In [255]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers

pd.set_option('display.max_rows', 500)

In [297]:
# Cleaning Data

df = pd.read_csv('data/stock_XY_train.csv')
print('OG Data Size:{}'.format(df.shape))

df = df[df.columns[df.isnull().mean() < 0.3]] # TO-DO: Tinker around with mean threshold.
df = df.dropna()
print('New Data Size:{}'.format(df.shape))
del df['operatingProfitMargin'] # Got rid of this column because it is all `1`. No reason to keep.

del df['Ticker']
del df['Sector']

# dropping for now cuz this hot encoding is annoying. 
# TO-DO: Get hot encoding working for Sector + Ticker columns
# pd.get_dummies(df, columns = ['Sector', 'Ticker'])
# print(list(df.columns))

OG Data Size:(12379, 226)
New Data Size:(6030, 188)


In [298]:
# Separating to train and test data
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

# Grabbing stats in order to normalize data
train_stats = train_df.describe()
train_stats.pop('Buy')
train_stats = train_stats.transpose()

# Separating labels
train_label = train_df.pop('Buy')
test_label = test_df.pop('Buy')

# Normalizing Data
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_df)
normed_test_data = norm(test_df)

In [299]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_df.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(
                    learning_rate=0.001, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-07, 
                    amsgrad=False,
                    name='Adam')

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'accuracy'])
    return model

model = build_model()
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 64)                11840     
_________________________________________________________________
dense_49 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 65        
Total params: 16,065
Trainable params: 16,065
Non-trainable params: 0
_________________________________________________________________


In [300]:
EPOCHS = 200

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,)

In [267]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,loss,mae,mse,accuracy,val_loss,val_mae,val_mse,val_accuracy,epoch
0,0.520347,0.552337,0.520347,0.502462,0.316236,0.501488,0.316236,0.538860,0
1,0.454838,0.495181,0.454838,0.584348,0.322664,0.491624,0.322664,0.583420,1
2,0.695899,0.498929,0.695899,0.600933,0.406425,0.490341,0.406425,0.604145,2
3,0.444578,0.466476,0.444578,0.644467,0.250883,0.456547,0.250883,0.619689,3
4,0.302687,0.444945,0.302687,0.670899,0.286613,0.472490,0.286613,0.622798,4
5,0.265774,0.431594,0.265774,0.683856,0.249688,0.451508,0.249688,0.644560,5
6,0.265209,0.420058,0.265209,0.698886,0.252444,0.457889,0.252444,0.646632,6
7,0.261960,0.420434,0.261960,0.703809,0.260153,0.440372,0.260153,0.638342,7
8,0.243129,0.411148,0.243129,0.706919,0.241719,0.443542,0.241719,0.640415,8
9,0.199984,0.395529,0.199984,0.718839,0.244681,0.438630,0.244681,0.663212,9


In [281]:
model.fit(normed_train_data, train_labels, epochs=200)

Train on 4824 samples
Epoch 1/200
4824/4824 [==============================] - 0s 38us/sample - loss: 0.0725 - mae: 0.1510 - mse: 0.0725 - accuracy: 0.9760
Epoch 2/200
4824/4824 [==============================] - 0s 37us/sample - loss: 0.1135 - mae: 0.1504 - mse: 0.1135 - accuracy: 0.9768
Epoch 3/200
4824/4824 [==============================] - 0s 38us/sample - loss: 0.0392 - mae: 0.1467 - mse: 0.0392 - accuracy: 0.9768
Epoch 4/200
4824/4824 [==============================] - 0s 39us/sample - loss: 0.0362 - mae: 0.1401 - mse: 0.0362 - accuracy: 0.9801
Epoch 5/200
4824/4824 [==============================] - 0s 37us/sample - loss: 0.0341 - mae: 0.1349 - mse: 0.0341 - accuracy: 0.9813
Epoch 6/200
4824/4824 [==============================] - 0s 39us/sample - loss: 0.0356 - mae: 0.1378 - mse: 0.0356 - accuracy: 0.9820
Epoch 7/200
4824/4824 [==============================] - 0s 40us/sample - loss: 0.0404 - mae: 0.1397 - mse: 0.0404 - accuracy: 0.9795
Epoch 8/200
4824/4824 [=================

4824/4824 [==============================] - 0s 37us/sample - loss: 0.0292 - mae: 0.1217 - mse: 0.0292 - accuracy: 0.9878
Epoch 62/200
4824/4824 [==============================] - 0s 37us/sample - loss: 0.0275 - mae: 0.1198 - mse: 0.0275 - accuracy: 0.9886
Epoch 63/200
4824/4824 [==============================] - 0s 38us/sample - loss: 0.0255 - mae: 0.1155 - mse: 0.0255 - accuracy: 0.9890
Epoch 64/200
4824/4824 [==============================] - 0s 36us/sample - loss: 0.0243 - mae: 0.1137 - mse: 0.0243 - accuracy: 0.9905
Epoch 65/200
4824/4824 [==============================] - 0s 41us/sample - loss: 0.0263 - mae: 0.1139 - mse: 0.0263 - accuracy: 0.9909
Epoch 66/200
4824/4824 [==============================] - 0s 36us/sample - loss: 0.0243 - mae: 0.1148 - mse: 0.0243 - accuracy: 0.9913
Epoch 67/200
4824/4824 [==============================] - 0s 36us/sample - loss: 0.0234 - mae: 0.1110 - mse: 0.0234 - accuracy: 0.9917
Epoch 68/200
4824/4824 [==============================] - 0s 37us/sa

4824/4824 [==============================] - 0s 39us/sample - loss: 0.0182 - mae: 0.0986 - mse: 0.0182 - accuracy: 0.9959
Epoch 122/200
4824/4824 [==============================] - 0s 39us/sample - loss: 0.0189 - mae: 0.1009 - mse: 0.0189 - accuracy: 0.9952
Epoch 123/200
4824/4824 [==============================] - 0s 40us/sample - loss: 0.0179 - mae: 0.0974 - mse: 0.0179 - accuracy: 0.9946
Epoch 124/200
4824/4824 [==============================] - 0s 40us/sample - loss: 0.0179 - mae: 0.0973 - mse: 0.0179 - accuracy: 0.9950
Epoch 125/200
4824/4824 [==============================] - 0s 36us/sample - loss: 0.0202 - mae: 0.1022 - mse: 0.0202 - accuracy: 0.9930
Epoch 126/200
4824/4824 [==============================] - 0s 39us/sample - loss: 0.0193 - mae: 0.0994 - mse: 0.0193 - accuracy: 0.9946
Epoch 127/200
4824/4824 [==============================] - 0s 47us/sample - loss: 0.0219 - mae: 0.1079 - mse: 0.0219 - accuracy: 0.9936
Epoch 128/200
4824/4824 [==============================] - 0s 

4824/4824 [==============================] - 0s 52us/sample - loss: 0.0150 - mae: 0.0894 - mse: 0.0150 - accuracy: 0.9975
Epoch 182/200
4824/4824 [==============================] - 0s 55us/sample - loss: 0.0155 - mae: 0.0906 - mse: 0.0155 - accuracy: 0.9965
Epoch 183/200
4824/4824 [==============================] - 0s 52us/sample - loss: 0.0153 - mae: 0.0902 - mse: 0.0153 - accuracy: 0.9959
Epoch 184/200
4824/4824 [==============================] - 0s 52us/sample - loss: 0.0160 - mae: 0.0920 - mse: 0.0160 - accuracy: 0.9969
Epoch 185/200
4824/4824 [==============================] - 0s 53us/sample - loss: 0.0185 - mae: 0.0990 - mse: 0.0185 - accuracy: 0.9946
Epoch 186/200
4824/4824 [==============================] - 0s 51us/sample - loss: 0.0180 - mae: 0.0965 - mse: 0.0180 - accuracy: 0.9954
Epoch 187/200
4824/4824 [==============================] - 0s 52us/sample - loss: 0.0154 - mae: 0.0901 - mse: 0.0154 - accuracy: 0.9965
Epoch 188/200
4824/4824 [==============================] - 0s 

In [286]:
results = model.evaluate(normed_test_data, test_labels, batch_size=128)
print('test loss, test acc:', results)

1206/1206 [==============================] - 0s 10us/sample - loss: 0.5650 - mae: 0.5237 - mse: 0.5650 - accuracy: 0.6144
test loss, test acc: [0.5650229968044097, 0.5237288, 0.565023, 0.61442786]
